In [1]:
%matplotlib inline
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import spacy
from __future__ import unicode_literals
import ast # this is just use to evaluate the lemmas
import time
from sklearn.tree import DecisionTreeClassifier
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics import classification_report

In [2]:
## Training set found from somewhere (I'll find it in a min) 
## Classifications are under Sentiment: 0=neg and 1=pos
## SetimentText==Tweets

training_set = pd.read_csv("train.csv", index_col=False, encoding='latin-1', header=0)
training_set.columns

Index(['ItemID', 'Sentiment', 'SentimentText'], dtype='object')

In [ ]:
## Splitting the data for training and testing
## This assumes you have tokenized the data and it is included somehow

ts_new = training_set[:80000]
ts_test = training_set[~training_set.ItemID.isin(ts_new.ItemID)]

In [ ]:
vectorizer = TfidfVectorizer() # vectorizes and normalizes the data

features = vectorizer.fit_transform(ts_new.lemma.astype('unicode')) #assumes a sparse matrix, prepares features
#print(vectorizer.vocabulary_)
smatrix = vectorizer.transform(ts_rest.lemma.astype('unicode')) #puts test data in a form acceptable by the tree
#print(smatrix)

# these both utilize the training set
y = ts_new['Sentiment']
X = features

In [ ]:

# Sets up the decision tree
dt = DecisionTreeClassifier(min_samples_split=20, random_state=99)
t0 = time.time()

# Trains the tree
dtf = dt.fit(X,y)
t1 = time.time()

# Testing the trained tree with test data set
predict_dt = dtf.predict(smatrix)
t2 = time.time()

print('time to train '+str(t1-t0))
print('time to predict '+str(t2-t1))

print(classification_report(ts_rest.Sentiment, predict_dt))

In [ ]:
## Repeating some of the steps above to run the troll tweets through the tree

tweet_matrix = vectorizer.transform(tweets.lemma.astype('unicode'))

predict_tweets = dt.predict(tweet_matrix)
tweet_prob = dt.predict_proba(tweet_matrix)

In [ ]:
## looks at the features of the tree

plt.figure(figsize=(10,8))
plt.plot(dtf.feature_importances_, 'o')
#plt.ylim(0.005,0.051)
plt.show()

In [ ]:
## This is just looking at the feature importance and mapping it back to the words

# putting it back into pandas cuz I know how to work with that
b = dtf.feature_importances_.tolist()
out = pd.Series(b)
out = pd.DataFrame({'x':out.index, 'y':out.values})

# looking at the top features
out_cut = out[out.y>=.01]
print(out_cut)

# flips the vocab dict
maps = {value: key for key, value in vectorizer.vocabulary_.items()}

# pulls out the keywords based on x-index values
def get(key, d=maps, default=None):
    
    if key in d:
        return d[key]
    else:
        return default


abc = out_cut.x.apply(get)

# putting it together, I don't know why I called it abc...
abc = pd.DataFrame({'key':abc.index,'vocab':abc.values},index=out_cut.index)
abc['weight'] = out_cut.y.values